In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
import os
import zipfile
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import numpy as np

In [2]:
# Load the CSV data
data = pd.read_csv('/kaggle/input/biomaterial-attachement/BacteriaA.csv')

In [3]:
# List all image IDs from the unzipped images
image_ids = [int(file.split('.')[0]) for file in os.listdir("/kaggle/input/biomaterial-attachement/ProcessedImages")]

# Convert image IDs to a DataFrame
image_ids_df = pd.DataFrame(image_ids, columns=['FeatID'])

# Merge with the ground truth data
merged_data = pd.merge(image_ids_df, data, how='inner', on='FeatID')

In [4]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
#         self.transform = transform
        self.transform = transforms.Compose([transforms.Resize((200, 200)), transform])

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_id = self.dataframe.iloc[idx, 0]
        img_path = os.path.join(self.root_dir, f"{img_id}.png")
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        ground_truth = self.dataframe.iloc[idx, 1]
        ground_truth = torch.tensor(ground_truth).float()  # Ensure the ground truth is a float tensor

        return image, ground_truth

In [5]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Tanh(),
        )

    def forward(self, z, ground_truth):
        x = torch.cat((z, ground_truth), dim=-1)
        img = self.model(x)
        return img

In [6]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, img, ground_truth):
        x = torch.cat((img, ground_truth), dim=-1)
        validity = self.model(x)
        return validity

In [7]:
# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [8]:
# Create the dataset
dataset = CustomDataset(merged_data, "/kaggle/input/biomaterial-attachement/ProcessedImages", transform=transform)

In [9]:
# Create the dataloader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [40]:
# Training configuration
num_epochs = 500
batch_size = 32
learning_rate = 0.0001
latent_dim = 100  # Size of the latent space

# Create the Generator and Discriminator
generator = Generator(latent_dim + 1, 200 * 200 * 3)
discriminator = Discriminator(200 * 200 * 3 + 1)

# Use binary cross-entropy loss
adversarial_loss = nn.BCELoss()

# Use Adam optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

# Move the model and loss function to GPU if available
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")
generator = generator.to(device)
discriminator = discriminator.to(device)
adversarial_loss = adversarial_loss.to(device)

In [41]:
from torchvision.models import inception_v3
from scipy.linalg import sqrtm

def calculate_fid(act1, act2):
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
    ssdiff = np.sum((mu1 - mu2)**2.0)
    covmean = sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

def get_activations(images, model, batch_size=32, device='cuda'):
    model = model.to(device)
    model.eval()
    activations = []
    n_batches = len(images) // batch_size
    for i in range(n_batches):
        inputs = images[i * batch_size : (i+1) * batch_size].to(device)
        with torch.no_grad():
            preds = model(inputs).detach().cpu().numpy()
        activations.append(preds)
    activations = np.concatenate(activations, axis=0)
    return activations

inception_model = inception_v3(pretrained=True, transform_input=False, init_weights=False)
inception_model.fc = nn.Identity()  # Modify model to return the 2048 feature before the final fully connected layer

## Training Loop for FID Score

In [42]:
for epoch in range(num_epochs):
    for i, (imgs, labels) in enumerate(dataloader):
        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = torch.ones(batch_size, 1).to(device)
        fake = torch.zeros(batch_size, 1).to(device)

        # Configure input
        real_imgs = imgs.view(imgs.size(0), -1).to(device)
        labels = labels.view(labels.size(0), -1).to(device)

        # Train Generator

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = torch.randn(batch_size, latent_dim).to(device)

        # Generate a batch of images
        gen_imgs = generator(z, labels)

        # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        # Train Discriminator

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_pred = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(real_pred, valid)

        fake_pred = discriminator(gen_imgs.detach(), labels)
        d_fake_loss = adversarial_loss(fake_pred, fake)

        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

    print(f"[Epoch {epoch}/{num_epochs}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

[Epoch 0/500] [D loss: 0.02807108871638775] [G loss: 3.023982048034668]
[Epoch 1/500] [D loss: 0.005707713775336742] [G loss: 5.16981840133667]
[Epoch 2/500] [D loss: 0.035430748015642166] [G loss: 3.13159441947937]
[Epoch 3/500] [D loss: 0.0849471166729927] [G loss: 1.9864765405654907]
[Epoch 4/500] [D loss: 0.14974641799926758] [G loss: 1.5938005447387695]
[Epoch 5/500] [D loss: 0.207497239112854] [G loss: 1.6538164615631104]
[Epoch 6/500] [D loss: 0.18205046653747559] [G loss: 1.385213017463684]
[Epoch 7/500] [D loss: 0.10339033603668213] [G loss: 1.789929986000061]
[Epoch 8/500] [D loss: 0.09379064291715622] [G loss: 1.949488639831543]
[Epoch 9/500] [D loss: 0.06780929118394852] [G loss: 2.193237066268921]
[Epoch 10/500] [D loss: 0.03411285951733589] [G loss: 2.770676851272583]
[Epoch 11/500] [D loss: 0.09639646112918854] [G loss: 1.8662153482437134]
[Epoch 12/500] [D loss: 0.13100290298461914] [G loss: 1.5750457048416138]
[Epoch 13/500] [D loss: 0.054927852004766464] [G loss: 2.45

In [43]:
# After the training loop:
with torch.no_grad():
    # Sample a batch of real images from the dataset
    real_images, ground_truth = next(iter(dataloader))  # Just taking one batch for simplicity
    real_images = real_images.view(real_images.size(0), -1).to(device)
    ground_truth = ground_truth.view(ground_truth.size(0), -1).to(device)
    
    # Generate a batch of images using the trained generator
    noise = torch.randn(real_images.size(0), latent_dim).to(device)
    generated_images = generator(noise, ground_truth)
    
    # Reshape the images to 4D
    real_images = real_images.view(-1, 3, 200, 200)
    generated_images = generated_images.view(-1, 3, 200, 200)
    
    # Get activations
    real_activations = get_activations(real_images, inception_model, device=device)
    generated_activations = get_activations(generated_images, inception_model, device=device)
    
    # Calculate FID score between real_images and generated_images
    fid_score = calculate_fid(real_activations, generated_activations)

print(f"Final FID Score: {fid_score}")


Final FID Score: 1379.808374823088


In [20]:
# Move the model to evaluation mode
generator.eval()

# Generate a batch of noise vectors
z = torch.randn(batch_size, latent_dim).to(device)

# Define the ground truth for the generated images
ground_truth = torch.tensor([0.5]*batch_size).view(-1, 1).to(device)

# Generate a batch of images
gen_imgs = generator(z, ground_truth)

# Reshape and denormalize the images
gen_imgs = gen_imgs.view(gen_imgs.size(0), 3, 200, 200)  # Reshape
gen_imgs = gen_imgs * 0.5 + 0.5  # Denormalize

# Convert tensor to numpy array
gen_imgs = gen_imgs.detach().cpu().numpy()

# Now, `gen_imgs` is a numpy array containing your generated images


In [45]:
# import matplotlib.pyplot as plt

# # Plot the generated images
# fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(15, 6),
#                         subplot_kw={'xticks': [], 'yticks': []})

# for i, ax in enumerate(axs.flat):
#     img = gen_imgs[i].transpose(1, 2, 0)  # Transpose the image dimensions
#     ax.imshow(img)

# plt.tight_layout()
# plt.show()


In [14]:
torch.cuda.is_available()

True

In [16]:
torch.device("cuda:0")

device(type='cuda', index=0)

In [25]:
!pip install torchsummary

In [37]:
from torchsummary import summary

class GeneratorWrapper(nn.Module):
    def __init__(self, generator):
        super(GeneratorWrapper, self).__init__()
        self.generator = generator

    def forward(self, x):
        z, ground_truth = x.split([latent_dim, 1], dim=1)
        return self.generator(z, ground_truth)

class DiscriminatorWrapper(nn.Module):
    def __init__(self, discriminator):
        super(DiscriminatorWrapper, self).__init__()
        self.discriminator = discriminator

    def forward(self, x):
        img, ground_truth = x.split([200 * 200 * 3, 1], dim=1)
        return self.discriminator(img, ground_truth)

# Wrap your models
gen_wrapper = GeneratorWrapper(generator).to(device)
dis_wrapper = DiscriminatorWrapper(discriminator).to(device)

# Now you can call summary on the wrapped models
summary(gen_wrapper, (latent_dim + 1, ))
summary(dis_wrapper, (200 * 200 * 3 + 1, ))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]          13,056
              ReLU-2                  [-1, 128]               0
            Linear-3                  [-1, 256]          33,024
              ReLU-4                  [-1, 256]               0
            Linear-5               [-1, 120000]      30,840,000
              Tanh-6               [-1, 120000]               0
         Generator-7               [-1, 120000]               0
Total params: 30,886,080
Trainable params: 30,886,080
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 2.75
Params size (MB): 117.82
Estimated Total Size (MB): 120.57
----------------------------------------------------------------
----------------------------------------------------------------
        Layer (type)           

In [28]:
generator

Generator(
  (model): Sequential(
    (0): Linear(in_features=101, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=120000, bias=True)
    (5): Tanh()
  )
)

In [31]:
discriminator

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=120001, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [29]:
torch.save(generator, 'Base_GAN_model_G.pth')

In [30]:
torch.save(discriminator, 'Base_GAN_model_D.pth')